# Using Naive Bayes algorithm for spam detection

In this assigment, you will predict if a sms message is 'spam' or 'ham' (i.e. not 'spam') using the Bernoulli Naive Bayes *classifier*.

The training data is from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection.  Please download the zip file before running the code below. 


## Step 1:  Getting, understanding, and cleaning the dataset


###  Importing the dataset

In [1]:
# Import the usual libraries
import math
import numpy as np 
import pandas as pd  # To read in the dataset we will use the Panda's library
df = pd.read_table('SMSSpamCollection', sep = '\t', header=None, names=['label', 'sms_message'])

# Next we observe the first 5 rows of the data to ensure everything was read correctly
df.head() 

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Data preprocesssing
It would be more convenient if the labels were binary instead of 'ham' and 'spam'.  This way our code can always work with numerical values instead of strings.

In [2]:
df['label']=df.label.map({'spam':1, 'ham':0})
df.head() # Again, lets observe the first 5 rows to make sure everything worked before we continue

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Splitting the dcoument into training set and test set

In [3]:
# This time we will use sklearn's method for seperating the data
from sklearn.model_selection import train_test_split

df_train_msgs, df_test_msgs, df_ytrain, df_ytest = train_test_split(df['sms_message'],df['label'], random_state=0)

#Looking at the train/test split
print("The number of training examples: ", df_train_msgs.shape[0])
print("The number of test exampels: ", df_test_msgs.shape)

print("The first four labels")
print(df_ytrain[0:4])

print("The first four sms messages")
print(df_train_msgs[0:4])


The number of training examples:  4179
The number of test exampels:  (1393,)
The first four labels
872     0
831     1
1273    0
3314    0
Name: label, dtype: int64
The first four sms messages
872     Its going good...no problem..but still need li...
831     U have a secret admirer. REVEAL who thinks U R...
1273                                                Ok...
3314    Huh... Hyde park not in mel ah, opps, got conf...
Name: sms_message, dtype: object


###  Creating the feature vector from the text (feature extraction)

Each message will have its own feature vector.  For each message we will create its feature vector as we discussed in class; we will have a feature for every word in our vocabulary.  The $j$th feature is set to one ($x_j=1$) if the $j$th word from our vocabulary occurs in the message, and set the $j$ feature is set to $0$ otherwise ($x_j=0$).

We will use the sklearn method CountVectorize to create the feature vectors for every messge.

We could have written the code to do this directly by performing the following steps:
* remove capitalization
* remove punctuation 
* tokenize (i.e. split the document into individual words)
* count frequencies of each token 
* remove 'stop words' (these are words that will not help us predict since they occur in most documents, e.g. 'a', 'and', 'the', 'him', 'is' ...

In [4]:
# importing the library
from sklearn.feature_extraction.text import CountVectorizer
# creating an instance of CountVectorizer
# Note there are issues with the way CountVectorizer removes stop words.  To learn more: https://scikit-learn.org/stable/modules/feature_extraction.html#stop-words
vectorizer = CountVectorizer(binary = True, stop_words='english')

# If we wanted to perform Multnomial Naive Bayes, we would include the word counts and use the following code
#vectorizer = CountVectorizer(stop_words='english')

# To see the 'count_vector' object
print(vectorizer)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [5]:
# To see the 'stop words' 
print(vectorizer.get_stop_words())

frozenset({'very', 'side', 'twelve', 'well', 'our', 'inc', 'across', 'seeming', 'now', 'sixty', 'had', 'rather', 'third', 'without', 'both', 'whither', 'may', 'nowhere', 'everything', 'at', 'became', 'she', 'being', 'bill', 'whenever', 'empty', 'about', 'show', 'last', 'alone', 'have', 'everyone', 'interest', 'thus', 'fifteen', 'de', 'would', 'but', 'latter', 'noone', 'though', 'anything', 'yet', 'seem', 'are', 'system', 'sometimes', 'cry', 'never', 'former', 'together', 'beside', 'hence', 'itself', 'due', 'he', 'my', 'back', 'detail', 'less', 'am', 'among', 'six', 'out', 'through', 'it', 'any', 'few', 'whereas', 'not', 'besides', 'on', 'as', 'only', 'fire', 'further', 'amount', 'is', 'everywhere', 'whereafter', 'myself', 'so', 'this', 'since', 'give', 'amongst', 'the', 'were', 'with', 'becoming', 'con', 'top', 'they', 'we', 'could', 'anyone', 'anyhow', 'when', 'always', 'his', 'wherever', 'move', 'behind', 're', 'here', 'another', 'who', 'co', 'front', 'other', 'four', 'sincere', 'how

In [6]:
# Create the vocabulary for our feature transformation
vectorizer.fit(df_train_msgs)

# Next we create the feature vectors for both the training data and the test data
X_train = vectorizer.transform(df_train_msgs).toarray() # code to turn the training emails into a feature vector
X_test = vectorizer.transform(df_test_msgs).toarray() # code to turn the test email into a feature vector

# Changing the target vectors data type  
y_train=df_ytrain.to_numpy() # Convereting from a Panda series to a numpy array
y_test = df_ytest.to_numpy()

# To observe what the data looks like 
print("The label of the first training example: ", y_train[0])
print("The first training example: ", X_train[0].tolist())# I needed to covernt the datatype to list so all the feature values would be printed

The label of the first training example:  0
The first training example:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Your code goes here

In [7]:
# You should NOT use the features of sklearn library in your code.

# Sepreate spam and ham emails
spam_x = []
ham_x = []
for i in range(len(y_train)):
    if y_train[i] == 1:
        spam_x.append(X_train[i])
    else:
        ham_x.append(X_train[i])

# Output 1: Estimation of P(y) (a)&(b)
P_spam = len(spam_x) / len(y_train)
P_ham = len(ham_x) / len(y_train)
print("P(1) = ", P_spam)
print("P(0) = ", P_ham)

P(1) =  0.13448193347690834
P(0) =  0.8655180665230916


In [8]:
def find_feature(word):
    msg = [word]
    vector = vectorizer.transform(msg).toarray()
    vec_list = vector.tolist()[0]

    index = vec_list.index(1)
    return index

print(find_feature("admirer"))
print(find_feature("secret"))

799
5642


In [49]:
# Probability of word x is choosen when email been labled spam or ham
def probability_x_withtype(index, sorh, m=0):
    check_list = []
    
    # we look for spam
    if sorh == 1:
        check_list = spam_x
    else:
        check_list = ham_x
    
    count_one = 0
    for i in range(len(check_list)):
        if check_list[i][index] == 1:
            count_one += 1
    
    p_spam = (count_one + m) / (len(check_list) + m * 2)
    return p_spam

# (c) & (d)
index_admirer = find_feature("admirer")
print("p(admirer|y) when y == 1: ", probability_x_withtype(index_admirer, 1))
print("p(admirer|y) when y == 0: ", probability_x_withtype(index_admirer, 0))

index_secret = find_feature("secret")
print("p(secret|y) when y == 1: ", probability_x_withtype(index_secret, 1))
print("p(secret|y) when y == 0: ", probability_x_withtype(index_secret, 0))

p(admirer|y) when y == 1:  0.014234875444839857
p(admirer|y) when y == 0:  0.0
p(secret|y) when y == 1:  0.014234875444839857
p(secret|y) when y == 0:  0.000552944429084877


In [50]:
# Output2: For each attribute x and each class y
attriX_spam = []
attriX_ham = []
for i in range(len(X_train[0])):
    attriX_spam.append(probability_x_withtype(i, 1))
    attriX_ham.append(probability_x_withtype(i, 0))
               
print(attriX_spam)
print(attriX_ham)    

[0.012455516014234875, 0.03558718861209965, 0.0, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0, 0.010676156583629894, 0.0017793594306049821, 0.0017793594306049821, 0.0035587188612099642, 0.0035587188612099642, 0.019572953736654804, 0.014234875444839857, 0.0017793594306049821, 0.0071174377224199285, 0.0035587188612099642, 0.0035587188612099642, 0.010676156583629894, 0.0035587188612099642, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0035587188612099642, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0035587188612099642, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0017793594306049821, 0.0035587188612099642, 0.0017793594306049821, 0.01601423487544484, 0.0035587

In [56]:
def test_email_spam(email, m=0):
    sum_log = 0
    if m == 0:
        use = attriX_spam
    else:
        use = attriX_spam_m
    for i in range(len(email)):
        with np.errstate(divide='ignore'):
            if email[i] == 1:
                sum_log += np.log(use[i])
            else:
                sum_log += np.log(1 - use[i])
    return np.log(P_spam) + sum_log   

In [57]:
def test_email_ham(email, m=0):
    sum_log = 0
    if m == 0:
        use = attriX_ham
    else:
        use = attriX_ham_m
    for i in range(len(email)):
        with np.errstate(divide='ignore'):
            if email[i] == 1:
                sum_log += np.log(use[i])
            else:
                sum_log += np.log(1 - use[i])
    return np.log(P_ham) + sum_log   

In [58]:
# (e): Predict first 5 examples
p_first5 = []
for i in X_test[:5]:
    if test_email_spam(i) >= test_email_ham(i):
        p_first5.append(1)
    else:
        p_first5.append(0)
p_first5 = np.array(p_first5)
print("My label of first 5 test examples: ", p_first5)
print("Real label of first 5 test examples: ",y_test[:5])

My label of first 5 test examples:  [0 0 0 0 0]
Real label of first 5 test examples:  [0 1 0 0 0]


In [59]:
# (f): Predict last 5 examples
p_last5 = []
for i in X_test[-5:]:
    if test_email_spam(i) >= test_email_ham(i):
        p_last5.append(1)
    else:
        p_last5.append(0)
p_last5 = np.array(p_last5)
print("My label of last 5 test examples: ", p_last5)
print("Real label of last 5 test examples: ",y_test[-5:])

My label of last 5 test examples:  [0 1 1 0 0]
Real label of last 5 test examples:  [0 1 1 0 0]


In [60]:
# Output 3: The predicted classes for the first 50 test examples.
p_50 = []
for i in X_test[:50]:
    if test_email_spam(i) >= test_email_ham(i):
        p_50.append(1)
    else:
        p_50.append(0)
p_50 = np.array(p_50)
print("My label of first 50 test examples:\n ", p_50)

My label of first 50 test examples:
  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 1 0 0 0 0 0 0 0 0 0 1]


In [44]:
# Output 4: Total number of test examples classified correctly.
count = 0
j = 0
for i in X_test[:]:
    if test_email_spam(i) >= test_email_ham(i) and y_test[j] == 1:
        count += 1
    if test_email_spam(i) < test_email_ham(i) and y_test[j] == 0:
        count += 1
    j += 1
print("Total number of test examples classified correctly: ", count)

Total number of test examples classified correctly:  1325


In [47]:
# Output 5: Total number of test examples classified incorrectly.
count_incorrect = len(y_test) - count
print("Total number of test examples classified incorrectly: ", count_incorrect)

Total number of test examples classified incorrectly:  68


In [48]:
# Output 6 & (g): The percentage error on the test examples.
error_percent = count_incorrect / len(y_test)
print("The percentage error on the test examples: ", error_percent)

The percentage error on the test examples:  0.048815506101938265


In [66]:
# (h): Try with smooth m = 0.2
attriX_spam_m = []
attriX_ham_m = []
for i in range(len(X_train[0])):
    attriX_spam_m.append(probability_x_withtype(i, 1, 0.2))
    attriX_ham_m.append(probability_x_withtype(i, 0, 0.2))
               
correct = 0
j = 0
for i in X_test[:]:
    if test_email_spam(i, 1) >= test_email_ham(i, 1) and y_test[j] == 1:
        correct += 1
    if test_email_spam(i, 1) < test_email_ham(i, 1) and y_test[j] == 0:
        correct += 1
    j += 1

new_error_percentage = (len(y_test) - correct) / len(y_test) 
print("New percentage error on the test examples: ", new_error_percentage)

New percentage error on the test examples:  0.014357501794687724


In [65]:
# (g): Try with Zero-R, since P(ham) > P(spam), we will always predict ham
zeroR_error = np.count_nonzero(y_test == 1) / len(y_test)
print("The error percentage for Zero-R is: ", zeroR_error)


The error percentage for Zero-R is:  0.13280689160086145
